In [ ]:
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain.chains import LLMChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.utilities import GoogleSearchAPIWrapper

## Memory in Agent

In order to add a memory to an agent we are going to perform the following steps:
1. We are going to create an `LLMChain` with memory.
2. We are going to use that `LLMChain` to create a **Custom Agent**.

In [ ]:
## Setting the Agent's Tools

search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

In [ ]:
## Setting Agent's Prompt

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools = tools,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["input", "chat_history", "agent_scratchpad"]
)

In [ ]:
## Setting the Memory Object

memory = ConversationBufferMemory(memory_key="chat_history")

Notice the usage of the `chat_history` variable in the `PromptTemplate`, which matches up with the dynamic key name in the `ConversationBufferMemory`.

In [ ]:
## Creating the Chain

llm_chain = LLMChain(
    llm = OpenAI(temperature=0),
    prompt = prompt
)

In [ ]:
## Creating the Agent and the Agent Chain

agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = tools,
    verbose = True
)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose = True
)

In [ ]:
agent_chain.run(input="How many people live in canada?")

In [ ]:
agent_chain.run(input="what is their national anthem called?")

## Message Memory in Agent backed by a Database

In order to add a memory with an external message store to an agent we are going to do the following steps:
1. We are going to create a `RedisChatMessageHistory` to connect to an external database to store the messages in.
2. We are going to create an `LLMChain` using that chat history as memory.
3. We are going to use that `LLMChain` to create a custom Agent.

In [ ]:
## Setting the Agent's Tools

search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

In [ ]:
## Setting the Agent's Prompt

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools = tools,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["input", "chat_history", "agent_scratchpad"]
)

In [ ]:
## Creating a `RedisChatMessageHistory` backed by the database

message_history = RedisChatMessageHistory(
    url="redis://localhost:6379/0", ttl=600, session_id="my-session"
)

In [ ]:
## Setting the Memory Object

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    chat_memory = message_history
)

In [ ]:
## Creating the Agent Chain

llm_chain = LLMChain(
    llm = OpenAI(temperature=0),
    prompt = prompt
)

agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = tools,
    verbose = True
)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose = True
)

In [ ]:
agent_chain.run(input="How many people live in canada?")

In [ ]:
agent_chain.run(input="what is their national anthem called?")